In [9]:
from sentence_transformers import SentenceTransformer, util

import pandas as pd
import numpy as np

# 데이터 불러오기, 파일 다운로드 후 경로 수정 필요
baemin_df = pd.read_csv("../guides/baemin.txt", sep="\n\n", names=['text'], engine='python')
baemin_df['text'] = baemin_df['text'].apply(lambda x: '[배민] ' + x)
baemin_guide = baemin_df['text'].to_list()
kakao_df = pd.read_csv("../guides/kakao.txt", sep="\n\n", names=['text'], engine='python')
kakao_df['text'] = kakao_df['text'].apply(lambda x: '[카톡] ' + x)
kakao_guide = kakao_df['text'].to_list()

# 가이드 리스트 합치기
guide_li = baemin_guide + kakao_guide

# 실질적으로 탑케이 필요함
embedder = SentenceTransformer("jhgan/ko-sbert-multitask")
corpus_embeddings = embedder.encode(guide_li, convert_to_tensor=True)


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [8]:

prompt = '아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. \
질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.\
\n\n### 질문 ###\n'

# Query sentences:
queries = [
    # '배달의민족에서 카카오페이 결제 하는 방법 알려줘.', 
    '카톡 사용 방법 알려줘', 
    # '치타가 들판을 가로 질러 먹이를 쫓는다.',
    # '카톡 이모티콘 무제한 어떻게 하는거야?',
    # '배민에서 결제를 어떻게 하는건지 모르겠어요',
    # '배민에서 배달비 어떻게 확인해?'
    ]

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5

# 유사도 점수 확인
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[:top_k]
    idxs = [x.item() for x in top_results]
    prompt += query

    for num, idx in enumerate(idxs):
        if 0 <= idx < len(guide_li):
            score = float("%.2f" % cos_scores[idx])

            if score >= 0.0:
                prompt += f'\n\n### 정보{num + 1} ###\n{score}\n{guide_li[idx].strip()}'
            else:
                break
print(prompt)

아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. 질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.

### 질문 ###
카톡 사용 방법 알려줘

### 정보1 ###
0.6
[카톡] 공감스티커 사용하기: 말하지 않고도 마음을 전달하는 카톡 프로필 공감스티커입니다. 요즘 내 일상과 기분을 색다르게 표현할 수 있는 카카오톡 프로필 꾸미기! 취향대로 꾸민 프로필에 공감스티커를 더해보세요. 내 프로필에 놀러 온 친구가 공감스티커를 눌러 마음을 전할 수 있어요. 오랜만에 업데이트된 친구의 프로필에 공감스티커를 꾹 눌러보세요! 굳이 톡 하지 않고 공감 스티커를 톡, 건드리는 것만으로도 반가운 마음을 전할 수 있겠죠? 1. 프로필 화면 아래 '프로필 편집' 버튼을 클릭해 공감스티커를 사용할 수 있습니다. 2. 다양한 스티커 중 원하는 것만 골라 내 프로필에 붙일 수 있습니다. 3. 내 프로필을 방문한 친구들이 보내준 공감 수만큼 숫자가 올라갑니다. * 친구들이 연속으로 공감할 수 있습니다. * 공감수/공감도가 프로필에 표시됩니다. * 공감한 친구 정보는 나만 알 수 있습니다. * 공감스티커를 길게 꾹 누르면 취소할 수도 있습니다.

### 정보2 ###
0.58
[카톡] 다양한 방법으로 친구 추가하기: QR코드부터 카톡 ID까지, 다양한 방식으로 친구를 추가하고 간편하게 관리할 수 있습니다. 새로운 관계를 맺을 때마다 필요한 카카오톡 친구 추가. 꼭 전화번호를 주고받지 않더라도 카카오톡 친구가 될 수 있다는 사실 알고 계셨나요? 카톡 친구를 추가하는 다양한 방법 중 나에게 가장 편한 방법을 골라보세요. 1. 친구탭에서 친구 추가 버튼을 선택합니다. 2. 상대방에게 나의 카톡 QR코드를 보여주세요. 상대방이 카메라로 QR코드를 인식하면 나의 카톡 친구로 추